In [1]:
# Import modules

from google.cloud import bigquery
from google.oauth2 import service_account

import sqlite3

import pandas as pd
import pandas_gbq

In [4]:
# Establish pathway to Google BigQuery (GBQ) account
service_path = 
service_file = 
gbq_proj_id = 
gbq_dataset_id = 

private_key = service_path + service_file

# Get credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# Creat client to connect with GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [2]:
# Query for question 1 (sales by date by hour):

query_one = """
SELECT
  EXTRACT(DATE
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS date,
  EXTRACT(HOUR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS hour,
  SUM(total) AS total_spend,
  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as num_trans,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as num_items
FROM
  `evident-catcher-327918.wedge_master.transArchive*`
WHERE department != 0 and
      department != 15 and
      trans_status != 'M' and
      trans_status != 'C' and
      trans_status != 'J' and
      (trans_status = '' or 
      trans_status = ' ' or 
      trans_status = 'V' or 
      trans_status = 'R')
GROUP BY
  date,
  hour
ORDER BY
  date ASC,
  hour ASC
"""

In [5]:
# Send query to GBQ and export results as pandas data frame

df_one = (
    client.query(query_one)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (and create) database
db = sqlite3.connect("20211018_connor_wedge.db")

# Export pandas dataframe to database
df_one.to_sql("sales_date_hour", db, if_exists="replace")

# Close database
db.close()

In [6]:
# Query for question 2 (sales by owner by month):

query_two = """
SELECT
  card_no,
  EXTRACT(YEAR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS year,
  EXTRACT(MONTH
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS month,
  SUM(total) AS sales,
  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items
FROM
  `evident-catcher-327918.wedge_master.transArchive*`
WHERE department != 0 and
      department != 15 and
      trans_status != 'M' and
      trans_status != 'C' and
      trans_status != 'J' and
      (trans_status = '' or 
      trans_status = ' ' or 
      trans_status = 'V' or 
      trans_status = 'R')
GROUP BY
  card_no,
  year,
  month
ORDER BY
  card_no,
  year ASC,
  month ASC
"""

In [7]:
# Send query to GBQ and export results as pandas data frame
df_two = (
    client.query(query_two)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (or create) database
db = sqlite3.connect("20211018_connor_wedge.db")

# Export pandas dataframe to database
df_two.to_sql("sales_owner_year_month", db, if_exists="replace")

# Close database
db.close()

In [ ]:
# This creates the empty department lookup table in GBQ, which is needed for the below query
# which joins with it to create the department name column.

table_id = "dept_lookup"
location_name = f"{gbq_proj_id}.{gbq_dataset_id}.{table_id}"

client.create_table(location_name)

In [ ]:
# Opening local .csv file with department lookup info in it, and importing data to GBQ table
# Note: dept_num is cast as a float, since that is its type in the transaction files.


filename = "dept_lookup.csv"

dataset_ref = client.dataset(gbq_dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True
job_config.schema = [
    bigquery.SchemaField("dept_num", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("dept_name", "STRING", mode="NULLABLE")]


with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)


In [8]:
# Query for question 3 (sales by product by year by month):

query_three = """
SELECT
  upc,
  description,
  department,
  dept_name,
  EXTRACT(YEAR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS year,
  EXTRACT(MONTH
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(datetime))) AS month,
  SUM(total) AS sales,
  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items
FROM
  `evident-catcher-327918.wedge_master.transArchive*` as trans
LEFT JOIN `evident-catcher-327918.wedge_master.dept_lookup` as dept ON dept.dept_num = trans.department
WHERE department != 0 and
      department != 15 and
      trans_status != 'M' and
      trans_status != 'C' and
      trans_status != 'J' and
      (trans_status = '' or 
      trans_status = ' ' or 
      trans_status = 'V' or 
      trans_status = 'R')
GROUP BY
  upc,
  description,
  department,
  dept_name,
  year,
  month
ORDER BY
  description,
  year ASC,
  month ASC
"""

In [9]:
# Send query to GBQ and export results as pandas data frame

df_three = (
    client.query(query_three)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (or create) database
db = sqlite3.connect("20211018_connor_wedge.db")

# Export pandas dataframe to database
df_three.to_sql("sales_product_year_month", db, if_exists="replace")

# Close database
db.close()